# Классификация текстовых документов посредством нейронных сетей


<hr>

С.Ю. Папулин (papulin.study@yandex.ru)

### Содержание

- [Предобработка данных](#Предобработка-данных)
- [Классификация текстовых документов](#Классификация-текстовых-документов)
    - [Загрузка исходных данных](#Загрузка-исходных-данных)
    - [Наивный байесовский классификатор](#Наивный-байесовский-классификатор)
    - [Полносвязная нейронная сеть](#Полносвязная-нейронная-сеть)
- [Рекуррентная нейронная сеть](#Рекуррентная-нейронная-сеть)
    - [LSTM](#LSTM)
    - [Embedding](#Embedding)
    - [Embedding и LSTM](#Embedding-и-LSTM)
    - [Сверточная нейронная сеть](#Сверточная-нейронная-сеть)

Подключение библиотек:

In [ ]:
import numpy as np

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
import matplotlib.pyplot as plt
%matplotlib inline

Команды установки `TensorFlow`:
```bash
pip install --upgrade pip
pip install tensorflow==2.10.0
```

Подключение пакетов

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

In [ ]:
# pip install --upgrade tensorflow-hub

## Предобработка данных

#### Обработка естественного языка с `TextVectorization`

#### Обработка структурированных данных

Слои для обработки категориальных признаков:
- `StringLookup` 
- `IntegerLookup`
- `Hashing` 
- `CategoryEncoding`

Слои для обработки числовых призгаков:
- `Normalization`
- `Discretization`

#### Обработка изображений

Стандартизация:
- `Resizing`
- `CenterCrop`
- `Rescaling`

Аугментация:
- `RandomCrop`
- `RandomFlip`
- `RandomTranslation`
- `RandomZoom`
- `RandomRotation`
- `RandomHeight`
- `RandomWidth`
- `RandomContrast`

## Классификация текстовых документов

In [ ]:
RANDOM_STATE = 100

### Загрузка исходных данных

In [ ]:
from sklearn.datasets import fetch_20newsgroups

In [ ]:
data = fetch_20newsgroups(
    subset="all", 
    shuffle=True, 
    remove=("headers", "footers", "quotes"), 
    random_state=123
)

In [ ]:
names = np.array(data.target_names)
names

In [ ]:
topics = (names=='comp.graphics')\
    | (names=='comp.os.ms-windows.misc')\
    | (names=='comp.sys.ibm.pc.hardware')\
    | (names=='comp.sys.mac.hardware')\
    | (names=='comp.windows.x')

In [ ]:
topic_labels = np.where(topics)[0]
topic_labels

In [ ]:
y = np.where(np.isin(data.target, topic_labels), 1, 0)
y[:5]

In [ ]:
np.unique(y, return_counts=True)

In [ ]:
# Формирование тестового множества
X_train, X_test, y_train, y_test = train_test_split(
    data.data, y, test_size=0.3, random_state=RANDOM_STATE
)

In [ ]:
# Формирование проверочного множества
X_train_, X_val, y_train_, y_val = train_test_split(
    X_train, y_train, test_size=0.3, random_state=RANDOM_STATE
)

### Наивный байесовский классификатор

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
NUM_FEATURES = 10000

In [ ]:
vectorizer = TfidfVectorizer(lowercase=True, stop_words="english",
                             use_idf=False, ngram_range=(1,1),
                             max_features=NUM_FEATURES,
                             smooth_idf=True)                        
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
vocab = list(vectorizer.vocabulary_.keys())
vocab[:4]

In [ ]:
tokenizer = vectorizer.build_tokenizer()
analyzer = vectorizer.build_analyzer()

In [ ]:
# Средняя длина текста
np.mean(list(map(lambda x: len(tokenizer(x)), X_train)))

In [ ]:
# Средняя длина текста с учетом словаря
np.mean(list(map(lambda x: len(analyzer(x)), X_train)))

In [ ]:
# Создание и обучение модели
m_multNB = MultinomialNB(alpha=0.1).fit(X_train_tfidf, y_train)

# Предсказания для тестового подмножества
# y_test_pred = m_multNB.predict(X_val_tfidf)

# Доля правильных классификаций на тестовом подмножестве
print("Accuracy =", m_multNB.score(X_test_tfidf, y_test))
print("Precision =", precision_score(m_multNB.predict(X_test_tfidf), y_test))

### Полносвязная нейронная сеть

In [ ]:
vectorizer_layer = layers.TextVectorization(
    output_mode="tf_idf",
    max_tokens=NUM_FEATURES
)

vectorizer_layer.adapt(X_train_)

In [ ]:
vectorizer_layer.get_vocabulary()[:5]

In [ ]:
def build_model():
    model = models.Sequential()
    model.add(vectorizer_layer)
    model.add(layers.Dense(128, activation="relu", input_shape=(NUM_FEATURES,)))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam",
                  loss="binary_crossentropy",
                  metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Precision()])
    return model

In [ ]:
# Построение модели
model = build_model()

# Описание модели
model.summary()

# Обучение
train_history = model.fit(
    X_train_, y_train_.tolist(), 
    epochs=15, 
    validation_data=(X_val, y_val.tolist()),
    batch_size=50,
    verbose=1
)

In [ ]:
INDX = 0

# Построение графиков ошибок обучения
plt.figure(figsize=[14, 4])

epochs = np.arange(1, len(train_history.history["loss"])+1)

plt.subplot(1,2,1)  # кросс-энтропия
plt.plot(epochs[INDX:], train_history.history["loss"][INDX:], "-og", label="train")
plt.plot(epochs[INDX:], train_history.history["val_loss"][INDX:], "-o", color="orange", label="val")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.grid(True)
plt.legend()

plt.subplot(1,2,2)  # доля правильных классификаций
plt.plot(epochs[INDX:], train_history.history["binary_accuracy"][INDX:], "-og", label="train")
plt.plot(epochs[INDX:], train_history.history["val_binary_accuracy"][INDX:], "-o", color="orange", label="val")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.grid(True)
plt.legend()

plt.show()

In [ ]:
# Выбираем количество эпох и заново обучаем сеть на всём обучающем множестве
best_num_epochs = 5

# Построение модели
model = build_model()

# Обучение
train_history = model.fit(X_train, y_train.tolist(),
                          epochs=best_num_epochs, 
                          batch_size=50,
                          verbose=1)

# Оценка качества модели
_, train_error__acc, train_error__prec = model.evaluate(X_train, y_train.tolist())
_, test_error__acc, test_error__prec = model.evaluate(X_test, y_test.tolist())

print("Train:\n")
print("\tAccuracy = \t", train_error__acc)
print("\tPrecision = \t", train_error__prec)
print("Test:\n")
print("\tAccuracy = \t", test_error__acc)
print("\tPrecision = \t", test_error__prec)

## Рекуррентная нейронная сеть

Преобразование текстовых документов в векторный вид

In [ ]:
NUM_FEATURES = 10000
TEXT_LENGTH = 20

In [ ]:
# Преобразование текста в последовательность индексов слов
# одной длины с добавлением нулей, если количество токенов 
# меньше выборанного значения
vectorizer_layer = layers.TextVectorization(
    output_sequence_length=TEXT_LENGTH,
    output_mode="int",
    max_tokens=NUM_FEATURES,
    standardize="lower_and_strip_punctuation"
)

In [ ]:
vectorizer_layer.adapt(X_train_)

In [ ]:
vectorizer_layer.vocabulary_size()

In [ ]:
vectorizer_layer.get_vocabulary()[:5]

In [ ]:
X_train_tokens_padded = vectorizer_layer(X_train_)
X_val_tokens_padded = vectorizer_layer(X_val)
X_train_tokens_padded[:1]

### LSTM

In [ ]:
NUM_ENCODER_FEATURES = vectorizer_layer.vocabulary_size()

#### One-Hot преобразование

В рекуррентную сеть необходимо подавать токены в виде вектора. Простейший вариант это представить токен как вектор размера `NUM_FEATURES` (размер словаря), в котором элемент с индексом токена равен  `1`, а все остальные `0`.

In [ ]:
def one_hot(x):
    # tranform text to sequence of indices
    outputs_seq_ = tf.reshape(vectorizer_layer([x,])[0], shape=(-1,1))
    features_indx = tf.reshape(tf.range(0, TEXT_LENGTH, dtype="int64"), shape=(-1,1))
    index_to_update = tf.concat((features_indx, outputs_seq_), axis=1)
    # one-hot encoding
    outputs_one_hot = tf.zeros((TEXT_LENGTH, NUM_ENCODER_FEATURES))
    outputs_one_hot = tf.tensor_scatter_nd_update(outputs_one_hot, index_to_update, tf.ones(TEXT_LENGTH))
    # remove 1's from the 1st column
    first_index = tf.reshape(tf.zeros(TEXT_LENGTH, dtype="int64"), shape=(-1,1))
    index_to_remove = tf.concat((features_indx, first_index), axis=1)
    return tf.tensor_scatter_nd_update(outputs_one_hot, index_to_remove, tf.zeros(TEXT_LENGTH))

In [ ]:
X_train_[0]

In [ ]:
one_hot(X_train_[0])

In [ ]:
BATCH_SIZE = 200

In [ ]:
train_X_dataset = tf.data.Dataset.from_tensor_slices(X_train_).map(one_hot)
train_y_dataset = tf.data.Dataset.from_tensor_slices(y_train_)
train_dataset = (
    tf.data.Dataset.zip((train_X_dataset, train_y_dataset))
        .batch(BATCH_SIZE)
        .prefetch(tf.data.AUTOTUNE)
)

val_X_dataset = tf.data.Dataset.from_tensor_slices(X_val).map(one_hot)
val_y_dataset = tf.data.Dataset.from_tensor_slices(y_val)
val_dataset = (
    tf.data.Dataset.zip((val_X_dataset, val_y_dataset))
        .batch(BATCH_SIZE)
        .prefetch(tf.data.AUTOTUNE)
)

In [ ]:
def build_model():
    model = models.Sequential()
    model.add(layers.Masking(mask_value=0., input_shape=(TEXT_LENGTH, NUM_ENCODER_FEATURES)))
    model.add(layers.LSTM(units=64, input_shape=(TEXT_LENGTH, NUM_ENCODER_FEATURES)))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam",
                  loss="binary_crossentropy",
                  metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Precision()])
    return model

In [ ]:
# Построение модели
model = build_model()

# Описание модели
model.summary()

# Обучение
train_history = model.fit(
    train_dataset, 
    epochs=5, 
    batch_size=100,
    validation_data=val_dataset,
    verbose=1
)

In [ ]:
# TODO: test

### Embedding

In [ ]:
BATCH_SIZE = 200

In [ ]:
train_X_dataset = tf.data.Dataset.from_tensor_slices(X_train_)
train_y_dataset = tf.data.Dataset.from_tensor_slices(y_train_)
train_dataset = (
    tf.data.Dataset.zip((train_X_dataset, train_y_dataset))
        .batch(BATCH_SIZE)
        .prefetch(tf.data.AUTOTUNE)
)

val_X_dataset = tf.data.Dataset.from_tensor_slices(X_val)
val_y_dataset = tf.data.Dataset.from_tensor_slices(y_val)
val_dataset = (
    tf.data.Dataset.zip((val_X_dataset, val_y_dataset))
        .batch(BATCH_SIZE)
        .prefetch(tf.data.AUTOTUNE)
)

In [ ]:
def build_model():
    model = models.Sequential()
    model.add(vectorizer_layer)
    model.add(layers.Embedding(
        input_length=TEXT_LENGTH,
        input_dim=NUM_ENCODER_FEATURES,
        output_dim=128,
        mask_zero=True))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam",
                  loss="binary_crossentropy",
                  metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Precision()])
    return model

In [ ]:
# Построение модели
model = build_model()

# Описание модели
model.summary()

# Обучение
train_history = model.fit(
    train_dataset, 
    epochs=10, 
    batch_size=100,
    validation_data=val_dataset,
    verbose=1
)

In [ ]:
# TODO: test

### Embedding и LSTM

In [ ]:
def build_model():
    model = models.Sequential()
    model.add(vectorizer_layer)
    model.add(layers.Embedding(
        input_dim=NUM_ENCODER_FEATURES,
        input_length=TEXT_LENGTH,
        output_dim=128,
        mask_zero=True))
    model.add(layers.Bidirectional(layers.LSTM(32)))
    model.add(layers.Dense(32, activation="relu"))
    model.add(layers.Dense(1,  activation="sigmoid"))
    model.compile(optimizer="adam",
                  loss="binary_crossentropy",
                  metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Precision()])
    return model

In [ ]:
# Построение модели
model = build_model()

# Описание модели
model.summary()

# Обучение
train_history = model.fit(
    train_dataset, 
    epochs=10, 
    batch_size=50,
    validation_data=val_dataset,
    verbose=1
)

In [ ]:
# TODO: test

### Сверточная нейронная сеть

In [ ]:
def build_model():
    model = models.Sequential()
    model.add(vectorizer_layer)
    model.add(layers.Embedding(
        input_length=TEXT_LENGTH,
        input_dim=NUM_ENCODER_FEATURES,
        output_dim=256,
        mask_zero=True))
    model.add(layers.Conv1D(128, kernel_size=5, activation="relu"))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(32, activation="relu"))
    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam",
                  loss="binary_crossentropy",
                  metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Precision()])
    return model

In [ ]:
# Построение модели
model = build_model()

# Описание модели
model.summary()

# Обучение
train_history = model.fit(
    train_dataset, 
    epochs=10, 
    batch_size=100,
    validation_data=val_dataset,
    verbose=1
)

In [ ]:
# TODO: test